# Nireports

Nireports is a framework to easily combine what we call reportlets, that is data visualizations that help assess the quality of a particular processing step, into HTML visual report. In this notebook, we give you an example about how to build your own visual report using the nireports framework. The goal with this example visual report is to summarize some group properties of functional connectivity (FC) matrices computed from a single dataset in order to assess the quality of connectivity extraction.


The first step is to generate the visualizations you would like and store them in a file with a BIDS-compatible name.

In [ ]:
# Load data
import pandas as pd

# Define the path where the derivatives are stored
output = ("/home/cprovins/derivatives/hcph/")
# Load fMRI duration after censoring
good_timepoints_df = pd.read_csv("./data/fMRI_duration_after_censoring.csv")

A reportlet does not have to be stored in an image format, it is also possible to incorporate HTML chunks into the overall report. Let us look at an example together. The goal with the next reportlet is to verify whether each session has enough timepoints after censoring to reliably estimate FC. If not, that session needs to be identified and excluded. We use plotly to generate an interactive plot in which hovering on the points tells you the session it corresponds to. The interactivity would not be possible with an image that is why we store the reportlet in HTML format.

In [ ]:
import plotly.offline as pyo
# The literature indicates that less than 5 minutes of fMRI signal after censoring is not enough to reliably estimate functional connectivity.
DURATION_CUT_OFF = 300
# Generate each reportlets
def group_report_censoring(good_timepoints_df, output) -> None:
    """
    Generate a group report about censoring.

    This function generates an HTML report that includes an interactive scatterplot
    showing the fMRI duration after censoring. The scatterplot includes
    error bars for the confidence interval and a red line indicating a duration cutoff.

    Parameters:
    -----------
    good_timepoints_df: pd.Dataframe
        A DataFrame containing information the fMRI duration after censoring.
    output : str
        Path to the output directory
    """
    filenames = good_timepoints_df["filename"]
    durations = good_timepoints_df["duration"]

    # Constructing the data for the plot
    # Add jitter to x values
    jitter = 0.2  # adjust this value to change the amount of jitter
    x_values = [1 + np.random.uniform(-jitter, jitter) for _ in range(len(durations))]
    data = [
        {
            "x": x_values,
            "y": durations,
            "text": filenames,
            "mode": "markers",
            "type": "scatter",
            "hoverinfo": "text",
            "marker": {"opacity": 0.5},
        }
    ]

    # Adding a red line at 5 minutes
    red_line = {
        "type": "line",
        "x0": 0,
        "y0": DURATION_CUT_OFF,
        "x1": 1.5,
        "y1": DURATION_CUT_OFF,
        "line": {"color": "red", "width": 3, "dash": "dashdot"},
    }

    # Layout settings
    layout = {
        "hovermode": "closest",
        "title": "Duration of fMRI signal after censoring",
        "yaxis": {"title": "Duration [s]"},
        "xaxis": {"showticklabels": False, "range": [0.5, 1.5]},
        "shapes": [red_line],
        "width": 600,
        "height": 600,
        "font": {"size": 16},
        "annotations": [
            {
                "x": 0.8,
                "y": DURATION_CUT_OFF - DURATION_CUT_OFF / 55,
                "xref": "x",
                "yref": "y",
                "text": f"QC cutoff of {DURATION_CUT_OFF/60} min",
                "showarrow": False,
                "font": {"color": "red"},
            }
        ],
    }

    fig = {"data": data, "layout": layout}

    # Save the plot as an HTML file
    pyo.plot(
        fig,
        filename=op.join(output, "reportlets", "group_desc-censoring_bold.html"),
        auto_open=False,
    )
group_report_censoring(good_timepoints_df, output)
group_reportlet_fc_dist(fc_matrices, output)
qc_fc_dict = group_reportlet_qc_fc(fc_matrices, iqms_df, output)